In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from scipy.spatial.distance import pdist, squareform
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers.legacy import Adam, Adagrad
np.set_printoptions(precision=3,suppress=True)
tf.keras.backend.set_floatx('float32')
print(tf.__version__)

In [ ]:
import NNNN_source as nn

In [ ]:
dir_name = os.getcwd() + '/data/'
cars = pd.read_csv(dir_name+'cars_df5.csv')
cars['price'] = np.log(cars['price'])
coords = cars.groupby(['location_id','lat', 'long']).size().index.to_frame().values
dist_square = np.float32(squareform(pdist(coords[:,1:])) ** 2)

In [ ]:
N = np.shape(cars)[0]
# p = 30 # + income, D1, D2, z0

X =  cars[[name for name in cars.columns if name not in  ['price','lat', 'long', 'model_id','location_id']]].copy()
y =  cars['price'].copy()
z0 = cars['model_id'].copy()
zL = cars['location_id'].copy()
Z0 = np.array(pd.get_dummies(z0), dtype=np.float32)
ZL = np.array(pd.get_dummies(zL), dtype=np.float32)

In [ ]:
def make_mean_model():

    input_X  = Input(shape=(np.shape(X_train)[1],),  dtype='float32')
    input_Z0 = Input(shape=(np.shape(Z0_train)[1],), dtype='float32')
    input_ZL = Input(shape=(np.shape(ZL_train)[1],), dtype='float32')
    
    m   = Dense(10, activation='relu')(input_X)
    m   = Dense(3, activation='relu')(m)
    
    xb  = Dense(1, activation='linear')(m)    
    zv0 = Dense(1, activation='linear', use_bias=False)(input_Z0)
    zvL = Dense(1, activation='linear', use_bias=False)(input_ZL)

    mean_model = Model(inputs=[input_X, input_Z0, input_ZL], outputs=[xb, zv0, zvL])
    return mean_model

In [ ]:
batch_size = 1024
pretrain = 150
patience = 50
max_epochs = 500
moments_epochs = 50
epsilon = 1e-3

lr_mean, lr_disp = 0.0005, 0.1
optimizer = Adam(learning_rate = lr_mean)

phi_init, lam0_init, lamL_init, lsq_init = 0.5, 0.5, 0.5, 0.1
phi_type, lamL_type = 'mme', 'mme'

In [ ]:
MSE, phi, lam0, lamL, lsq = [np.zeros(5) for _ in range(5)]
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mu_test_dict, y_test_dict = {}, {}
for i, (train_index, test_index) in enumerate(kf.split(y)):

    X_tv, X_test = np.array(X.loc[train_index].copy(), dtype=np.float32), np.array(X.loc[test_index].copy(), dtype=np.float32)
    y_tv, y_test = np.array(y[train_index], dtype=np.float32), np.array(y[test_index], dtype=np.float32)
    Z0_tv, Z0_test = np.array(Z0[train_index], dtype=np.float32), np.array(Z0[test_index], dtype=np.float32)
    ZL_tv, ZL_test = np.array(ZL[train_index], dtype=np.float32), np.array(ZL[test_index], dtype=np.float32)
    
    X_train, X_valid, y_train, y_valid, Z0_train, Z0_valid, ZL_train, ZL_valid = train_test_split(
        X_tv, y_tv, Z0_tv, ZL_tv, test_size=0.1, random_state=42)
    
    N_train = np.shape(y_train)[0]
    train_batch = tf.data.Dataset.from_tensor_slices((X_train, Z0_train, ZL_train, y_train)).shuffle(N_train).batch(batch_size)
    
    K.clear_session()
    nn.seed_everything()

    M = make_mean_model()
    res = nn.train_model_RBF2(
        M, train_batch, [X_train, Z0_train, ZL_train, y_train], [X_valid, Z0_valid, ZL_valid, y_valid], dist_square,
        optimizer, lr_disp, phi_init, lam0_init, lamL_init, lsq_init, phi_type, lamL_type, epsilon,
        pretrain, moments_epochs, max_epochs, patience,
        adjust=False, wts_init=None, verbose=True, seed=0)
    
    mu_test = np.sum(M([X_test, Z0_test, ZL_test]), axis=0).T
    MSE[i]  = np.mean((y_test-mu_test)**2)
    phi[i]  = res['phi']
    lam0[i] = res['lam0']
    lamL[i] = res['lamL']
    lsq[i]  = res['lsq']
        
    mu_test_dict[str(i)] = mu_test.T
    y_test_dict[str(i)]  = y_test
    
print('MSE : ', np.round(np.mean(MSE),3), np.round(np.std(MSE),3))
print('phi : ', np.round(np.mean(phi),3), np.round(np.std(phi),3))
print('lam0: ', np.round(np.mean(lam0),3), np.round(np.std(lam0),3))
print('lamL: ', np.round(np.mean(lamL),3), np.round(np.std(lamL),3))
print('lsq : ', np.round(np.mean(lsq),3), np.round(np.std(lsq),3))

In [ ]:
y_test_concat = np.concatenate((y_test_dict['0'], y_test_dict['1'],y_test_dict['2'],y_test_dict['3'],y_test_dict['4']))
mu_test_concat = np.concatenate((mu_test_dict['0'], mu_test_dict['1'],mu_test_dict['2'],mu_test_dict['3'],mu_test_dict['4']))
out = pd.DataFrame()
out['y'] = y_test_concat
out['mu'] = mu_test_concat
plt.plot(y_test_dict['0'], mu_test_dict['0'], 'b.', markersize=0.2)